In [1]:
import argparse
import json
import re
import GraphDB.utils as utils
from GraphDB.LegalGraphDB import LegalGraphDB
import os, sys
from typing import Dict
from tqdm import tqdm

"""
hierarchy relation 생성

- clause graph의 REFERS_TO는 이미 구축되어 있는 것이라 가정

python make_hierarchy.py
"""
# config.json 파일 경로를 절대 경로로 설정
config_path = f"./configs/config.json"

print(f"Using config file: {config_path}")
with open(config_path, 'r') as f:
    config = json.load(f)

json_path = "../data/graph/clause/title_embedding/01/01_enforcement_main.json"
clause_type = "Clause_" + json_path.split('/')[-1].split('.')[0]
title_prefix = "Title_" + json_path.split('/')[-1].split('.')[0] + "_"

index_dict = {"편" : title_prefix+"doc", "장" : title_prefix+"chapter", 
              "절" : title_prefix+"section", "관" : title_prefix+"subsection", 
              "조" : clause_type, "항" : clause_type}


dbms = LegalGraphDB(auradb=False, config=config, json_path=json_path)


# json 불러오기
with open(json_path, "r", encoding='utf-8') as f:
    data = json.load(f)

print(len(data))


Using config file: ./configs/config.json
## Settings ##
## clause type :  Clause_01_enforcement_main
## title_prefix :  Title_01_enforcement_main_
## 서버와의 연결이 성공적으로 확인되었습니다.
1675


In [2]:
####### 전처리 단계 ############
# row 단위로 돌면서 확인

total_meta = []

for row in data:
    hierarchy = [title_prefix]

    for key, value in row['metadata']['title'].items():
        if value != None and key != 'supplementary':
            # 제7편 거래소 <개정 2013. 5. 28.> 같은 경우 전처리
            if re.search(r'<개정 \d{4}\. \d{1,2}\. \d{1,2}\.?>', value) != None:
                value = re.sub(r'<개정 \d{4}\. \d{1,2}\. \d{1,2}\.?>', '', value).strip()

            #  "제5장 온라인소액투자중개업자 등에 대한 특례 <신설 2015. 7. 24.>",
            if re.search(r'<신설 \d{4}\. \d{1,2}\. \d{1,2}\.?>', value) != None:
                value = re.sub(r'<신설 \d{4}\. \d{1,2}\. \d{1,2}\.?>', '', value).strip()

            hierarchy.append(value)

    # 삭제 항이 아닐 경우에만 & 부칙이 아닐 경우에만 추가
    if row['subtitle'] != None and row['metadata']['title']['supplementary'] == None:
        hierarchy.append(row['index'] + " " + row['subtitle'])
        total_meta.append(hierarchy)

print('## meta total length : ', len(total_meta))

# relationship 중복 제거 -> 청크 처리 필요
new_meta = []
for x in total_meta:
    if x in new_meta:
        print('## duplicated : ', x)
    if x not in new_meta:
        new_meta.append(x)

print('## after remove duplicate : ', len(new_meta))


## meta total length :  1657
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제2조 용어의 정의']
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제2조 용어의 정의']
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제6조제4항 집합투자의 적용배제']
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제6조제4항 집합투자의 적용배제']
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제7조제4항 금융투자업의 적용배제']
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제7조제4항 금융투자업의 적용배제']
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제7조제4항 금융투자업의 적용배제']
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제10조제3항 전문투자자의 범위 등']
## duplicated :  ['Title_01_enforcement_main_', '제1편 총칙', '제11조제1항 증권의 모집ㆍ매출']
## duplicated :  ['Title_01_enforcement_main_', '제2편 금융투자업', '제4장 영업행위 규칙', '제1절 공통 영업행위 규칙', '제1관 신의성실의무 등', '제47조제1항 본질적 업무의 범위 등']
## duplicated :  ['Title_01_enforcement_main_', '제2편 금융투자업', '제4장 영업행위 규칙', '제2절 금융투자업자별 영업행위 규칙', '제1관 투자매매업자 및 투자중개업자의 영업행위 규칙', '제68

### json 후처리 (<개정> 제거)

In [3]:
# ## json 후처리 -> '<(개정|신설) \d{4}\. \d{1,2}\. \d{1,2}\.?>' 없애기

# import re
# import json

# def postprocess_json(file_path):
#     # config_file 가져오기
#     with open(file_path, "r", encoding="utf-8") as f:
#         data = json.load(f)

#     # 정규 표현식 패턴 정의
#     patterns = re.compile(r'<(개정|신설) \d{4}\. \d{1,2}\. \d{1,2}\.?>')

#     # JSON 데이터 순회하며 패턴 제거
#     for item in data:
#         title = item.get("metadata", {}).get("title", {})
#         for key in title.keys():
#             if title[key]:
#                 title[key] = patterns.sub("", title[key])
#                 title[key] = title[key].strip()

#     # 결과 JSON 파일로 저장
#     with open(file_path, "w", encoding="utf-8") as file:
#         json.dump(data, file, ensure_ascii=False, indent=4)



In [4]:
# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/01/01_law_main.json"
# postprocess_json(file_path)

# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/01/01_enforcement_main.json"
# postprocess_json(file_path)

# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/01/01_order_main.json"
# postprocess_json(file_path)

# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/02/02_law_main.json"
# postprocess_json(file_path)

# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/02/02_enforcement_main.json"
# postprocess_json(file_path)

# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/02/02_order_main.json"
# postprocess_json(file_path)

# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/03/03_regulation_main.json"
# postprocess_json(file_path)

# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/08/08_enforcement_main.json"
# postprocess_json(file_path)

# # 파일 경로 설정
# file_path = "../data/graph/clause/title_embedding/08/08_law_main.json"
# postprocess_json(file_path)

### hierarchy 생성

In [5]:
dbms.create_hierarchy_graph(new_meta)

## node_type : root, law_index : None, name : Title_01_enforcement_main_, full_name : 
## node_type : Title_01_enforcement_main_doc, law_index : 제1편, name : 총칙, full_name : Title_01_enforcement_main_
## node_type : Clause_01_enforcement_main, law_index : 제1조, name : 목적, full_name : Title_01_enforcement_main_ 제1편 총칙
## clause node detected - reference :  Title_01_enforcement_main_ 제1편 총칙
## node_type : root, law_index : None, name : Title_01_enforcement_main_, full_name : 
## node_type : Title_01_enforcement_main_doc, law_index : 제1편, name : 총칙, full_name : Title_01_enforcement_main_
## node_type : Clause_01_enforcement_main, law_index : 제2조, name : 용어의 정의, full_name : Title_01_enforcement_main_ 제1편 총칙
## clause node detected - reference :  Title_01_enforcement_main_ 제1편 총칙
## node_type : root, law_index : None, name : Title_01_enforcement_main_, full_name : 
## node_type : Title_01_enforcement_main_doc, law_index : 제1편, name : 총칙, full_name : Title_01_enforcement_main_
## node_type : C

In [ ]:

    ########## 수정 코드 #############
    # 순회하면서 path 추가  



    ########### 그래프 생성 코드 #########
    # 생성된 total meta 2개씩 순회
    # for hierarchy_list in tqdm(total_meta):
    #     for idx in range(len(hierarchy_list)-1):
    #         # first node
    #         first_index = hierarchy_list[idx]

    #         # second node
    #         second_index = hierarchy_list[idx+1]
    #         first_node_type, first_law_index, first_name = get_node_info(first_index)
    #         second_node_type, second_law_index, second_name = get_node_info(second_index)

    #         # 첫 번재 노드가 존재하는지 확인
    #         first_node_id = dbms.get_node_id(first_node_type, "law_index", first_law_index, "name", first_name)

    #         if not first_node_id:
    #             # 노드 생성 - create_node
    #             first_node_id = dbms.create_node({"node_type" : first_node_type, "law_index" : first_law_index, "name" : first_name})
            
            
    #         # 두 번째 노드가 존재하는지 확인
    #         second_node_id = dbms.get_node_id(second_node_type, "law_index", second_law_index, "name", second_name)
    #         if not second_node_id:
    #             # 노드 생성 - create_node
    #             second_node_id = dbms.create_node({"node_type" : second_node_type, "law_index" : second_law_index, "name" : second_name})
            
    #         # id를 바탕으로 relationship 생성
    #         dbms.create_relationship("hierarchy", first_node_id, second_node_id)



# if __name__ == "__main__":
#     # config.json 파일 경로를 절대 경로로 설정
#     current_dir = os.path.dirname(os.path.abspath(__file__))
#     project_root = os.path.abspath(os.path.join(current_dir, '..'))

#     config_path = f"{project_root}/codes/configs/config.json"
#     # config_path = os.path.join(project_root, 'codes', 'configs', 'config.json')
#     print(f"Using config file: {config_path}")
#     with open(config_path, 'r') as f:
#         config = json.load(f)
    
#     main(config=config)


In [ ]:
# 